# Neural DSL on Google Colab - Complete Guide

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Lemniscate-SHA-256/Neural/blob/main/neural/cloud/examples/neural_colab_example.ipynb)

This notebook provides a comprehensive guide to using Neural DSL in Google Colab with GPU/TPU support, cloud optimizations, and advanced features.

## Table of Contents
1. [Setup and Installation](#setup)
2. [GPU/TPU Configuration](#gpu-tpu)
3. [Quick Start](#quick-start)
4. [Advanced Model Building](#advanced-models)
5. [Hyperparameter Optimization](#hpo)
6. [Model Visualization and Debugging](#visualization)
7. [Production Deployment](#deployment)
8. [Best Practices](#best-practices)
9. [Troubleshooting](#troubleshooting)
10. [Cleanup](#cleanup)

## 1. Setup and Installation <a id='setup'></a>

Install Neural DSL and required dependencies.

In [ ]:
# Install Neural DSL with full dependencies
!pip install -q git+https://github.com/Lemniscate-SHA-256/Neural.git

# Install cloud-specific dependencies
!pip install -q pyngrok

print("✓ Installation complete!")

## 2. GPU/TPU Configuration <a id='gpu-tpu'></a>

Check and configure GPU/TPU availability.

**Note:** To enable GPU in Colab, go to: Runtime → Change runtime type → Hardware accelerator → GPU

In [ ]:
# Check for GPU
!nvidia-smi

# Check TensorFlow GPU availability
import tensorflow as tf
print(f"\nTensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")
print(f"Built with CUDA: {tf.test.is_built_with_cuda()}")

### Initialize Cloud Executor

In [ ]:
from neural.cloud.cloud_execution import CloudExecutor
import json

# Initialize with Colab-optimized settings
executor = CloudExecutor(
    timeout=900,  # 15 minutes for longer training runs
    retry_attempts=3
)

# Display environment information
env_info = executor.get_environment_info()
print("Environment Configuration:")
print(json.dumps(env_info, indent=2, default=str))

print(f"\n{'='*50}")
print(f"Environment: {executor.environment}")
print(f"GPU Available: {executor.is_gpu_available}")
print(f"Optimization Level: {executor.optimization_level}/3")
print(f"{'='*50}")

## 3. Quick Start <a id='quick-start'></a>

Build and train your first model in 3 steps.

In [ ]:
# Step 1: Define model
simple_model = """
network SimpleCNN {
    input: (28, 28, 1)
    layers:
        Conv2D(32, (3, 3), "relu")
        MaxPooling2D((2, 2))
        Flatten()
        Dense(128, "relu")
        Dense(10, "softmax")
    loss: "categorical_crossentropy"
    optimizer: Adam(learning_rate=0.001)
}
"""

# Step 2: Compile
model_path = executor.compile_model(simple_model, backend='tensorflow')
print(f"✓ Model compiled to: {model_path}")

# Step 3: Train
results = executor.run_model(model_path, dataset='MNIST', epochs=2, batch_size=128)
if results['success']:
    print("✓ Training completed!")
else:
    print(f"✗ Training failed: {results['error']}")

## 4. Advanced Model Building <a id='advanced-models'></a>

### 4.1 CIFAR-10 CNN with Regularization

In [ ]:
cifar_model = """
network Cifar10Advanced {
    input: (32, 32, 3)
    layers:
        # First convolutional block
        Conv2D(64, (3, 3), "relu", padding="same")
        BatchNormalization()
        Conv2D(64, (3, 3), "relu", padding="same")
        BatchNormalization()
        MaxPooling2D((2, 2))
        Dropout(0.25)
        
        # Second convolutional block
        Conv2D(128, (3, 3), "relu", padding="same")
        BatchNormalization()
        Conv2D(128, (3, 3), "relu", padding="same")
        BatchNormalization()
        MaxPooling2D((2, 2))
        Dropout(0.25)
        
        # Third convolutional block
        Conv2D(256, (3, 3), "relu", padding="same")
        BatchNormalization()
        Conv2D(256, (3, 3), "relu", padding="same")
        BatchNormalization()
        GlobalAveragePooling2D()
        Dropout(0.25)
        
        # Dense layers
        Dense(512, "relu")
        BatchNormalization()
        Dropout(0.5)
        Dense(10, "softmax")
    loss: "categorical_crossentropy"
    optimizer: Adam(learning_rate=0.001)
}
"""

try:
    cifar_path = executor.compile_model(cifar_model, backend='tensorflow', validate=True)
    print(f"✓ CIFAR-10 model compiled: {cifar_path}")
    
    # Train with longer epochs
    cifar_results = executor.run_model(
        cifar_path,
        dataset='CIFAR10',
        epochs=5,
        batch_size=128,
        timeout=600
    )
    
    if cifar_results['success']:
        print("✓ CIFAR-10 training completed successfully!")
        print("\nTraining output (last 500 chars):")
        print(cifar_results['stdout'][-500:])
    else:
        print(f"✗ Training failed: {cifar_results['error']}")
        
except Exception as e:
    print(f"✗ Error: {e}")

### 4.2 Multi-Backend Comparison

In [ ]:
# Compile the same model to multiple backends
comparison_model = """
network ComparisonNet {
    input: (28, 28, 1)
    layers:
        Conv2D(32, (3, 3), "relu")
        MaxPooling2D((2, 2))
        Conv2D(64, (3, 3), "relu")
        MaxPooling2D((2, 2))
        Flatten()
        Dense(128, "relu")
        Dense(10, "softmax")
    loss: "categorical_crossentropy"
    optimizer: Adam(learning_rate=0.001)
}
"""

backends = ['tensorflow', 'pytorch']
backend_models = {}

print("Compiling to multiple backends...\n")
for backend in backends:
    try:
        path = executor.compile_model(comparison_model, backend=backend)
        backend_models[backend] = path
        print(f"✓ {backend.upper()}: {path}")
    except Exception as e:
        print(f"✗ {backend.upper()} failed: {e}")

print(f"\n✓ Successfully compiled to {len(backend_models)}/{len(backends)} backends")

## 5. Hyperparameter Optimization <a id='hpo'></a>

Use Neural DSL's HPO features to automatically find optimal hyperparameters.

In [ ]:
hpo_model = """
network HPOModel {
    input: (32, 32, 3)
    layers:
        Conv2D(HPO(choice(32, 64, 128)), (3, 3), "relu", padding="same")
        MaxPooling2D((2, 2))
        Dropout(HPO(range(0.1, 0.5, step=0.1)))
        
        Conv2D(HPO(choice(64, 128, 256)), (3, 3), "relu", padding="same")
        MaxPooling2D((2, 2))
        Dropout(HPO(range(0.1, 0.5, step=0.1)))
        
        Flatten()
        Dense(HPO(choice(128, 256, 512)), "relu")
        Dropout(0.5)
        Dense(10, "softmax")
    loss: "categorical_crossentropy"
    optimizer: Adam(learning_rate=HPO(log_range(0.0001, 0.01)))
    train {
        epochs: 10
        batch_size: HPO(choice(32, 64, 128))
        search_method: "bayesian"
        max_trials: 20
    }
}
"""

try:
    print("Setting up hyperparameter optimization...")
    hpo_path = executor.compile_model(hpo_model, backend='tensorflow')
    print(f"✓ HPO model compiled: {hpo_path}")
    
    print("\nStarting hyperparameter search (this may take several minutes)...")
    hpo_results = executor.run_model(
        hpo_path,
        dataset='CIFAR10',
        timeout=1800  # 30 minutes
    )
    
    if hpo_results['success']:
        print("✓ Hyperparameter optimization completed!")
        print("\nResults (last 1000 chars):")
        print(hpo_results['stdout'][-1000:])
    else:
        print(f"✗ HPO failed: {hpo_results['error']}")
        
except Exception as e:
    print(f"✗ HPO error: {e}")

## 6. Model Visualization and Debugging <a id='visualization'></a>

### 6.1 Visualize Model Architecture

In [ ]:
from IPython.display import Image, display

# Visualize the CIFAR model
try:
    viz_path = executor.visualize_model(cifar_model, output_format='png')
    print(f"✓ Visualization saved to: {viz_path}")
    display(Image(filename=viz_path))
except Exception as e:
    print(f"✗ Visualization failed: {e}")

### 6.2 Interactive Debugging with NeuralDbg

In [ ]:
from IPython.display import HTML, display

# Start debugging dashboard
try:
    print("Starting NeuralDbg dashboard...")
    dashboard = executor.start_debug_dashboard(
        cifar_model,
        backend='tensorflow',
        setup_tunnel=True,
        port=8050
    )
    
    if dashboard['status'] == 'running':
        print(f"✓ Dashboard is running!")
        print(f"Process ID: {dashboard['process_id']}")
        print(f"Local URL: {dashboard['dashboard_url']}")
        print(f"Public URL: {dashboard['tunnel_url']}")
        
        display(HTML(f"""
        <div style='padding: 20px; background-color: #f0f0f0; border-radius: 5px; margin: 10px 0;'>
            <h3>🔍 NeuralDbg Dashboard</h3>
            <p><a href='{dashboard['tunnel_url']}' target='_blank' style='color: #0066cc; font-size: 16px;'>
                Click here to open the dashboard
            </a></p>
        </div>
        """))
    else:
        print(f"✗ Dashboard failed: {dashboard.get('error', 'Unknown error')}")
        
except Exception as e:
    print(f"✗ Could not start dashboard: {e}")
    print("Note: ngrok tunnel requires authentication. Set up your ngrok auth token if needed.")

### 6.3 No-Code Interface

In [ ]:
# Start No-Code visual interface
try:
    print("Starting No-Code interface...")
    nocode = executor.start_nocode_interface(
        port=8051,
        setup_tunnel=True
    )
    
    if nocode['status'] == 'running':
        print(f"✓ No-Code interface is running!")
        print(f"Process ID: {nocode['process_id']}")
        
        display(HTML(f"""
        <div style='padding: 20px; background-color: #e8f4f8; border-radius: 5px; margin: 10px 0;'>
            <h3>🎨 Neural No-Code Interface</h3>
            <p><a href='{nocode['tunnel_url']}' target='_blank' style='color: #0066cc; font-size: 16px;'>
                Click here to open the visual builder
            </a></p>
        </div>
        """))
    else:
        print(f"✗ No-Code interface failed: {nocode.get('error', 'Unknown error')}")
        
except Exception as e:
    print(f"✗ Could not start No-Code interface: {e}")

## 7. Production Deployment <a id='deployment'></a>

Export models for production deployment.

In [ ]:
# Example: Save model for deployment
import os

deployment_model = """
network ProductionModel {
    input: (224, 224, 3)
    layers:
        Conv2D(64, (3, 3), "relu", padding="same")
        MaxPooling2D((2, 2))
        Conv2D(128, (3, 3), "relu", padding="same")
        MaxPooling2D((2, 2))
        Conv2D(256, (3, 3), "relu", padding="same")
        GlobalAveragePooling2D()
        Dense(1000, "softmax")
    loss: "categorical_crossentropy"
    optimizer: Adam(learning_rate=0.001)
}
"""

# Compile for production
production_path = executor.compile_model(
    deployment_model,
    backend='tensorflow',
    output_file='/content/production_model.py'
)

print(f"✓ Production model saved to: {production_path}")
print(f"File size: {os.path.getsize(production_path)} bytes")

# Download the model
print("\nTo download the model:")
print("1. Click on the folder icon on the left")
print("2. Right-click on 'production_model.py'")
print("3. Select 'Download'")

## 8. Best Practices <a id='best-practices'></a>

In [ ]:
print("""\n=== Neural DSL on Colab: Best Practices ===\n
1. ✓ GPU Usage:
   - Enable GPU: Runtime → Change runtime type → GPU
   - Check availability before training
   - Use batch normalization for faster convergence

2. ✓ Memory Management:
   - Use appropriate batch sizes (64-128 for GPU)
   - Enable gradient checkpointing for large models
   - Clear GPU memory: tf.keras.backend.clear_session()

3. ✓ Training Efficiency:
   - Use mixed precision training on compatible GPUs
   - Leverage data augmentation
   - Save checkpoints regularly

4. ✓ Error Handling:
   - Always use try-except blocks
   - Set appropriate timeouts
   - Check model outputs before training

5. ✓ Cloud Optimizations:
   - Neural DSL automatically applies Colab-specific optimizations
   - GPU memory growth is enabled by default
   - Retry logic handles transient failures

6. ✓ Debugging:
   - Use NeuralDbg dashboard for real-time monitoring
   - Enable verbose logging when needed
   - Validate models before training

7. ✓ Cleanup:
   - Always run executor.cleanup() at the end
   - Close ngrok tunnels when done
   - Save important outputs to Google Drive
""")

## 9. Troubleshooting <a id='troubleshooting'></a>

In [ ]:
# Diagnostic tests
print("Running diagnostic tests...\n")

# Test 1: Environment detection
print("1. Environment Detection:")
print(f"   Detected: {executor.environment}")
print(f"   Expected: colab")
print(f"   Status: {'✓ PASS' if executor.environment == 'colab' else '✗ FAIL'}\n")

# Test 2: GPU availability
print("2. GPU Availability:")
print(f"   GPU Available: {executor.is_gpu_available}")
print(f"   Optimization Level: {executor.optimization_level}\n")

# Test 3: Model compilation
print("3. Model Compilation Test:")
test_model = """
network TestNet {
    input: (28, 28, 1)
    layers:
        Flatten()
        Dense(10, "softmax")
    loss: "categorical_crossentropy"
    optimizer: Adam()
}
"""
try:
    test_path = executor.compile_model(test_model, backend='tensorflow')
    print(f"   Status: ✓ PASS")
    print(f"   Output: {test_path}\n")
except Exception as e:
    print(f"   Status: ✗ FAIL")
    print(f"   Error: {e}\n")

# Test 4: Error handling
print("4. Error Handling Test:")
try:
    executor.compile_model("", backend='tensorflow')
    print(f"   Status: ✗ FAIL (should have raised error)\n")
except Exception:
    print(f"   Status: ✓ PASS (correctly caught empty DSL)\n")

print("Diagnostic tests complete!")

## 10. Cleanup <a id='cleanup'></a>

Clean up all resources before closing the notebook.

In [ ]:
# Comprehensive cleanup
print("Starting cleanup...\n")

# 1. Clean up executor resources
executor.cleanup()
print("✓ Executor resources cleaned")

# 2. Clear TensorFlow session
try:
    import tensorflow as tf
    tf.keras.backend.clear_session()
    print("✓ TensorFlow session cleared")
except:
    pass

# 3. Clear PyTorch cache (if available)
try:
    import torch
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print("✓ PyTorch CUDA cache cleared")
except:
    pass

print("\n✓ Cleanup complete!")
print("\nThank you for using Neural DSL!")

## Summary

This notebook covered:

1. ✓ **Setup**: Installation and configuration
2. ✓ **GPU/TPU**: Hardware acceleration setup
3. ✓ **Quick Start**: Build and train in 3 steps
4. ✓ **Advanced Models**: Complex architectures with regularization
5. ✓ **HPO**: Automatic hyperparameter optimization
6. ✓ **Visualization**: Model architecture and debugging tools
7. ✓ **Deployment**: Export for production
8. ✓ **Best Practices**: Tips for efficient cloud usage
9. ✓ **Troubleshooting**: Diagnostic tests and solutions
10. ✓ **Cleanup**: Proper resource management

### Next Steps

- 📚 Read the [documentation](https://github.com/Lemniscate-SHA-256/Neural)
- 🚀 Try more [examples](https://github.com/Lemniscate-SHA-256/Neural/tree/main/examples)
- 💬 Join the [community](https://github.com/Lemniscate-SHA-256/Neural/discussions)
- 🐛 Report [issues](https://github.com/Lemniscate-SHA-256/Neural/issues)

Happy modeling! 🎉